# **GLAUCOMA PREDICTION**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **IMPORTING LIBERARIES**

In [ ]:
#Importing the Libraries
import os
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.models import Sequential, model_from_json
from keras.layers import Conv2D, Dense, Dropout, BatchNormalization, Flatten, MaxPool2D, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.applications.densenet import DenseNet121
import tensorflow as tf


# **PRE-PROCESSING**

In [ ]:
# Defining a function to Crop and change the image from RGB to Gray Scale
def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol

        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
    #         print(img1.shape,img2.shape,img3.shape)
            img = np.stack([img1,img2,img3],axis=-1)
    #         print(img.shape)
        return img

#Defining the Path 
path = [ '/content/drive/MyDrive/NOGlaucoma_CROP','/content/drive/MyDrive/severe glaucomaCropped','/content/drive/MyDrive/moderatecrop']
images = []
labels = []
for n,i in enumerate(path):
    for j in tqdm(os.listdir(i)):
        img_path = os.path.join(i,j)
        img = cv2.imread(img_path)
        img = crop_image_from_gray(img,tol=7)
        img = cv2.resize(img, (224,224))
        images.append(img)
        labels.append(n)

#Creating Array of Images and Lables
images = np.array(images)/255
labels = np.array(labels)


100%|██████████| 571/571 [00:11<00:00, 48.15it/s]


# **CNN ALGORITHM**

In [ ]:
# Test Train Split
images, labels = shuffle(images, labels, random_state=32)
x_train, x_valid, y_train, y_valid = train_test_split(images, labels, test_size=0.20, random_state=32)
#x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.15, random_state=40)

#Model
datagen = ImageDataGenerator(zoom_range=0.2, horizontal_flip=True, vertical_flip=True,
                            fill_mode='constant', cval=0.)
train_gen = datagen.flow(x_train, y_train, batch_size=32)

tr = DenseNet121(weights='imagenet', include_top=False, input_shape=(224,224,3))

model = Sequential()
model.add(tr)
model.add(GlobalAveragePooling2D())

model.add(Dense(25, activation='sigmoid'))

model.add(Dense(80,activation='sigmoid'))

model.add(Dense(100,activation='sigmoid'))

model.add(Dense(50,activation='sigmoid'))

model.add(Dropout(0.8))

model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
ck = ModelCheckpoint('CNN2Weights2.h5', monitor='val_loss', save_best_only=True, mode='auto', verbose=1)
re = ReduceLROnPlateau(monitor='val_loss', mode='auto', factor=0.2, patience=4, verbose=1)
model.summary()

history = model.fit_generator(train_gen, epochs=50, steps_per_epoch=800//32,
                              verbose=1, validation_data=(x_valid,y_valid),
                              callbacks=[ck,re])

# pred = model.evaluate(x_valid,y_valid)
# print('Test Accuracy:', pred[1]*100)

#Saving the Model
model.save("CNN2Weights2.h5")

29097984/29084464 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 global_average_pooling2d (G  (None, 1024)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 25)                25625     
                                                                 
 dense_1 (Dense)             (None, 80)                2080      
                                                                 
 dense_2 (Dense)             (None, 100)               8100      
                                                                 
 dense_3 (Dense)             (None, 50)                50

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/50
25/25 [==============================] - ETA: 0s - loss: 0.3974 - accuracy: 0.3350
Epoch 1: val_loss improved from inf to 0.14547, saving model to CNN2Weights2.h5
25/25 [==============================] - 45s 715ms/step - loss: 0.3974 - accuracy: 0.3350 - val_loss: 0.1455 - val_accuracy: 0.3362 - lr: 0.0010
Epoch 2/50
25/25 [==============================] - ETA: 0s - loss: -0.1831 - accuracy: 0.3831
Epoch 2: val_loss improved from 0.14547 to 0.09735, saving model to CNN2Weights2.h5
25/25 [==============================] - 15s 585ms/step - loss: -0.1831 - accuracy: 0.3831 - val_loss: 0.0973 - val_accuracy: 0.3362 - lr: 0.0010
Epoch 3/50
25/25 [==============================] - ETA: 0s - loss: -0.1861 - accuracy: 0.3584
Epoch 3: val_loss did not improve from 0.09735
25/25 [==============================] - 11s 450ms/step - loss: -0.1861 - accuracy: 0.3584 - val_loss: 0.0991 - val_accuracy: 0.3362 - lr: 0.0010
Epoch 4/50
25/25 [==============================] - ETA: 0s - loss: 

# **TESTING THE ALGORITHM**

***Predicting Moderate Glaucoma***

In [ ]:
# Path for New Image to be Tested On
p = '/content/drive/MyDrive/moderatecrop/m45 - Copy (2).png'

im = load_img(p ,target_size = (224,224))

i = img_to_array(im)
input_arr = np.array([i])
input_arr.shape
#Predicting New Image's Category of Glaucoma
prediction =((model.predict(input_arr))+1)
prediction_lable = np.around(prediction)

In [ ]:
#Printing the Predicted Result
if prediction >= 0 and prediction < 1:
  print('the patient has No Glaucoma')
elif prediction >= 1 and prediction < 1.99 :
  print('the patient has Severe Glaucoma')
else:
  print('patient has Moderate Glaucoma')

patient has Moderate Glaucoma


***Predicting Severe Glaucoma***

In [ ]:
# Path for second Image to be Tested On
p = '/content/drive/MyDrive/severe glaucomaCropped/severe38 - Copy (10) - Copy - Copy - Copy.png'

im = load_img(p ,target_size = (224,224))

i = img_to_array(im)
input_arr = np.array([i])
input_arr.shape
#Predicting New Image's Category of Glaucoma
prediction =(model.predict(input_arr))
prediction_lable = np.around(prediction)

In [ ]:
#Printing the Predicted Result
if prediction_lable >= 0 and prediction_lable < 1:
  print('the patient has No Glaucoma')
elif prediction_lable >= 1 and prediction_lable < 2 :
  print('the patient has Severe Glaucoma')
else:
  print('patient has Moderate Glaucoma')

the patient has Severe Glaucoma


***Predicting No Glaucoma***

In [ ]:
# Path for second Image to be Tested On
p = '/content/drive/MyDrive/NOGlaucoma_CROP/P766186_20190313_145319_Color_L_001 - Copy (13) - Copy.JPG'

im = load_img(p ,target_size = (224,224))

i = img_to_array(im)
input_arr = np.array([i])
input_arr.shape
#Predicting New Image's Category of Glaucoma
prediction =(model.predict(input_arr))

(1, 224, 224, 3)

In [ ]:
#Printing the Predicted Result
if prediction >= 0 and prediction < 1:
  print('the patient has No Glaucoma')
elif prediction >= 1 and prediction < 2 :
  print('the patient has Severe Glaucoma')
else:
  print('patient has Moderate Glaucoma')

the patient has No Glaucoma
